<a href="https://colab.research.google.com/github/ooutaksaluk/MBT-analysis/blob/main/Main_Battle_Tank_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



**Main Battle Tank analysis**



In [2]:
!pip install anvil-uplink opencv-python numpy torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os

train_dir = '/content/drive/My Drive/dataset/train'
validation_dir = '/content/drive/My Drive/dataset/validation'

# ตรวจสอบว่าโฟลเดอร์มีอยู่จริง
if os.path.exists(train_dir):
    print("พบโฟลเดอร์:", train_dir)
else:
    print("ไม่พบโฟลเดอร์:", train_dir)


พบโฟลเดอร์: /content/drive/My Drive/dataset/train


**เตรียมข้อมูลและการโหลดข้อมูลด้วย Keras**

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # ใช้ 20% เป็น validation
)

# โหลดข้อมูล Train (ใช้ 80%)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

# โหลดข้อมูล Validation (ใช้ 20%)
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)



Found 72 images belonging to 2 classes.
Found 18 images belonging to 2 classes.


**Training Model**

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# สร้างโมเดล CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='sigmoid')  # 2 Output: Sight, Mantlet
])

# ใช้ binary_crossentropy เพราะเป็น Multi-Label (sigmoid)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.save('/content/drive/My Drive/dataset/trained_model.h5')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)





**Prediction**



In [8]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
import tensorflow as tf

# เชื่อม Google Drive (ถ้าต้องใช้ไฟล์จาก Drive)
from google.colab import drive
drive.mount('/content/drive')

# โหลดโมเดลที่ฝึกเสร็จแล้ว
model = tf.keras.models.load_model('/content/drive/My Drive/dataset/trained_model.h5')

# กำหนด label ที่ต้องการตรวจจับ
labels = ["Sight", "Mantlet"]

def predict_sight_mantlet(image_path):
    img = cv2.imread(image_path)

    if img is None:
        raise ValueError(f"ไม่พบไฟล์รูปภาพ: {image_path}")  # ตรวจสอบว่ารูปมีอยู่จริง

    img = cv2.resize(img, (150, 150))  # ปรับขนาดให้เหมาะสมกับ input ของโมเดล
    img_array = image.img_to_array(img)  # แปลงภาพเป็น array
    img_array = np.expand_dims(img_array, axis=0)  # เพิ่ม batch dimension
    img_array = img_array / 255.0  # ปรับ scale ให้อยู่ในช่วง [0,1]

    # ทำนายผล
    prediction = model.predict(img_array)[0]  # ดึงผลลัพธ์ออกมา

    # ตรวจสอบค่าความเป็นไปได้ของแต่ละ class
    predicted_labels = [labels[i] for i, value in enumerate(prediction) if value > 0.5]

    # แสดงผลลัพธ์
    return predicted_labels if predicted_labels else ["ไม่มีการตรวจพบ"]

# ทดสอบโมเดล (ใส่ path รูปจริง)
test_image_path = "/content/drive/My Drive/dataset/test_image.jpg"
result = predict_sight_mantlet(test_image_path)
print(f"ผลการทำนาย: {result}")



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step
ผลการทำนาย: ['Sight']


In [12]:
import anvil.server
import anvil.media

# เชื่อมต่อกับ Anvil Uplink
anvil.server.connect("server_LX2D76OV3RCR3XCSOMCFYQZL-JGQCERXRR5DWB5QH")

# โหลดโมเดลที่ฝึกเสร็จแล้ว
model = tf.keras.models.load_model('/content/drive/My Drive/dataset/trained_model.h5')


@anvil.server.callable
def process_image(file_bytes):
    # โหลดภาพจากไฟล์ที่อัปโหลด
    nparr = np.frombuffer(file_bytes, np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

    if img is None:
        raise ValueError("Error: ไม่สามารถอ่านไฟล์รูปภาพได้!")

    # ปรับขนาดภาพและ Normalize
    img_resized = cv2.resize(img, (150, 150)).astype("float32") / 255.0

    # ทำนายผล
    img_array = np.expand_dims(img_resized, axis=0)
    predictions = model.predict(img_array)[0]  # รับผลการจำแนก (Multi-Label)

    # ตรวจสอบว่าผลลัพธ์เป็นอาร์เรย์ที่ถูกต้อง
    if not isinstance(predictions, np.ndarray) or predictions.shape[0] != 2:
        raise ValueError("Error: โมเดลคืนค่าที่ไม่ถูกต้อง!")

    # กำหนด Label และ Bounding Box (ค่า Box จำเป็นต้องได้จาก Object Detection Model)
    labels = ["Sight", "Mantlet"]
    boxes = [(30, 30, 100, 100), (120, 50, 180, 120)]  # ตัวอย่าง Bounding Box

    # วาด Bounding Box ตามค่าที่โมเดลทำนาย
    for i, label in enumerate(labels):
        if predictions[i] > 0.5:  # ถ้าค่าความมั่นใจเกิน 0.5
            x1, y1, x2, y2 = boxes[i]  # พิกัด (จำเป็นต้องเปลี่ยนให้ตรงกับผลโมเดลจริง)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # แปลงภาพกลับเป็นไฟล์
    _, img_encoded = cv2.imencode('.jpg', img)
    return img_encoded.tobytes()

# รอรับคำสั่งจาก Anvil
anvil.server.wait_forever()

KeyboardInterrupt: 